<a href="https://colab.research.google.com/github/viggopatel/QM2-real/blob/main/Copy_of_EfW_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install Basemap
!pip install ipyleaflet

In [ ]:
%%capture
# Uninstall potentially conflicting libraries first
!pip uninstall -y pandas numpy matplotlib ipyleaflet geopy

# Reinstall core libraries, allowing pip to resolve versions
!pip install pandas numpy matplotlib ipyleaflet geopy

In [ ]:
# Uninstall potentially conflicting libraries first
# Using yes to confirm uninstall
!yes | pip uninstall -y pandas numpy matplotlib ipyleaflet geopy geopy

# Reinstall core libraries, allowing pip to resolve versions
# Combine installations to let pip handle dependencies
!pip install pandas numpy matplotlib ipyleaflet geopy==2.3.0 Basemap

# Optional: If you want to use %%capture again, add it back after verifying successful installation
# %%capture

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pylab

%matplotlib inline
pylab.rcParams['figure.figsize'] = (20, 15)

Found existing installation: pandas 2.3.0
Uninstalling pandas-2.3.0:
  Successfully uninstalled pandas-2.3.0
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: matplotlib 3.8.4
Uninstalling matplotlib-3.8.4:
  Successfully uninstalled matplotlib-3.8.4
Found existing installation: ipyleaflet 0.19.2
Uninstalling ipyleaflet-0.19.2:
  Successfully uninstalled ipyleaflet-0.19.2
Found existing installation: geopy 2.3.0
Uninstalling geopy-2.3.0:
  Successfully uninstalled geopy-2.3.0
  Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached numpy-2.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached ipyleaflet-0.19.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached geopy-2.3.0-py3-none-any.whl.metadata (6.7 kB)
  Usin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EfW3.csv')

In [ ]:
!pip install geopy==2.3.0

from geopy.geocoders import Nominatim

In [ ]:
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False)

df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce')


In [ ]:
from ipyleaflet import Map, CircleMarker, basemaps, Marker, DivIcon # Import Marker and DivIcon
import pandas as pd

# Calculate center using grouped data (this is no longer needed for centering on the UK, but keeping it for potential future use or if you still want the data's center as a reference)
# center = (df['Lat'].mean(), df['Long'].mean())

# Set the center and zoom for the UK
uk_center = (55.0, -3.0) # Approximate center of the UK
uk_zoom = 5           # Adjust zoom level as needed (e.g., 5 or 6)

# Create the map with a different base layer
# Using CARTO's Positron map which is minimalist and grayscale/light
m = Map(center=uk_center, zoom=uk_zoom, basemap=basemaps.CartoDB.Positron)

# Ensure 'Capacity' is numeric after removing commas.
# The errors='coerce' in the previous pd.to_numeric call will result in NaN for invalid entries.
# We keep it as float to allow for NaN values.
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False).astype(float)

for index, row in df.iterrows():
    # Check if the 'Capacity' value is a valid number (not NaN) and if 'Name' exists
    if pd.notna(row['Capacity']) and 'Name' in row and pd.notna(row['Name']):
        # Calculate the radius, ensuring the result is a valid number before converting to int
        # Also ensure the result is non-negative, as radius must be positive
        calculated_radius = row['Capacity'] / 75 # Use 1000 as divisor again
        if calculated_radius >= 0:
            # Add the CircleMarker
            marker = CircleMarker(
                location=(row['Lat'], row['Long']),
                # Convert to integer only if the calculated radius is a valid non-negative number
                radius=int(calculated_radius),
                color=(row['Colour']),
                fill_color=(row['Colour']),
                fill_opacity=1
            )
            m.add_layer(marker)

            # Add a label using DivIcon for the 'Name' column
            # Add 'color: black;' to the style
            label = Marker(
                location=(row['Lat'], row['Long']),
                icon=DivIcon(
                    html=f'<div style="font-size: 10px; white-space: nowrap; color: black;">{row["Name"]}</div>', # Basic styling with black text
                    # Adjust icon_anchor to position the label relative to the location
                    # [0, 0] anchors the top-left of the div at the location
                    # [0, -10] would anchor above the location by 10 pixels
                    icon_anchor=[0, 0]
                )
            )
            m.add_layer(label)

        else:
             # Optional: Handle cases where calculated_radius is negative (shouldn't happen with positive capacity but good practice)
             print(f"Skipping row {index} due to negative calculated radius: {calculated_radius}")
    else:
        # Optional: Handle rows where Capacity or Name is invalid
        print(f"Skipping row {index} due to invalid Capacity or Name: Capacity={row.get('Capacity')}, Name={row.get('Name')}")


display(m)

Map(center=[55.0, -3.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
from ipyleaflet import Map, CircleMarker, basemaps
import pandas as pd

# Calculate center using grouped data (this is no longer needed for centering on the UK, but keeping it for potential future use or if you still want the data's center as a reference)
# center = (df['Lat'].mean(), df['Long'].mean())

# Set the center and zoom for the UK
uk_center = (55.0, -3.0) # Approximate center of the UK
uk_zoom = 5            # Adjust zoom level as needed (e.g., 5 or 6)
height=7000

# Create the map with a different base layer
# Using CARTO's Positron map which is minimalist and grayscale/light
m = Map(center=uk_center, zoom=uk_zoom, basemap=basemaps.CartoDB.Positron, height=700)

# Ensure 'Capacity' is numeric after removing commas.
# The errors='coerce' in the previous pd.to_numeric call will result in NaN for invalid entries.
# We keep it as float to allow for NaN values.
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False).astype(float)

for index, row in df.iterrows():
    # Check if the 'Capacity' value is a valid number (not NaN)
    if pd.notna(row['Capacity']):
        # Calculate the radius, ensuring the result is a valid number before converting to int
        # Also ensure the result is non-negative, as radius must be positive
        # Adjusted divisor based on previous context
        calculated_radius = row['Capacity'] / 75 # Use 1000 as divisor again
        if calculated_radius >= 0:
            marker = CircleMarker(
                location=(row['Lat'], row['Long']),
                # Convert to integer only if the calculated radius is a valid non-negative number
                radius=int(calculated_radius),
                color=(row['Colour']),
                fill_color=(row['Colour']),
                fill_opacity=1
            )
            m.add_layer(marker)
        else:
             # Optional: Handle cases where calculated_radius is negative (shouldn't happen with positive capacity but good practice)
             print(f"Skipping row {index} due to negative calculated radius: {calculated_radius}")
    else:
        # Optional: Handle rows where Capacity is not a valid number (NaN)
        print(f"Skipping row {index} due to invalid Capacity: {row['Capacity']}")


display(m)


Map(center=[55.0, -3.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
# prompt: How can I download the above

# You need to have a file named 'my_notebook.ipynb' or similar in your file system.
# The code above is from the output cells of the notebook, not the input cells.
# To download a notebook from Google Colab, you can use the 'File' menu, then select 'Download', and choose either '.ipynb' or '.py'.
# If you want to download a file that was created or modified within the notebook's execution, you can use the `files.download` function.

# Example of how to download a file created in the notebook:
# Assuming you saved a dataframe to a csv file named 'output_data.csv' earlier in your code
df.to_csv('output_data.csv', index=False)
from google.colab import files
files.download('output_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ipyleaflet import Map, CircleMarker, basemaps, Marker, DivIcon # Import Marker and DivIcon
import pandas as pd

# Calculate center using grouped data (this is no longer needed for centering on the UK, but keeping it for potential future use or if you still want the data's center as a reference)
# center = (df['Lat'].mean(), df['Long'].mean())

# Set the center and zoom for the UK
uk_center = (55.0, -3.0) # Approximate center of the UK
uk_zoom = 5            # Adjust zoom level as needed (e.g., 5 or 6)

# Create the map with a different base layer
# Using CARTO's Positron map which is minimalist and grayscale/light
m = Map(center=uk_center, zoom=uk_zoom, basemap=basemaps.CartoDB.Positron)

# Ensure 'Capacity' is numeric after removing commas.
# The errors='coerce' in the previous pd.to_numeric call will result in NaN for invalid entries.
# We keep it as float to allow for NaN values.
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False).astype(float)

for index, row in df.iterrows():
    # Check if the 'Capacity' value is a valid number (not NaN) and if 'Name' exists
    if pd.notna(row['Capacity']) and 'Name' in row and pd.notna(row['Name']):
        # Calculate the radius, ensuring the result is a valid number before converting to int
        # Also ensure the result is non-negative, as radius must be positive
        # Adjusted divisor based on previous context
        calculated_radius = row['Capacity'] / 75 # Use 1000 as divisor again
        if calculated_radius >= 0:
            # Add the CircleMarker
            marker = CircleMarker(
                location=(row['Lat'], row['Long']),
                # Convert to integer only if the calculated radius is a valid non-negative number
                radius=int(calculated_radius),
                color=(row['Colour']),
                fill_color=(row['Colour']),
                fill_opacity=1
            )
            m.add_layer(marker)

            # Add a label using DivIcon for the 'Name' column
            # Add 'color: black;' to the style and ensure background is transparent/none
            label = Marker(
                location=(row['Lat'], row['Long']),
                icon=DivIcon(
                    html=f'<div style="font-size: 10px; white-space: nowrap; color: black; background: none !important; background-color: transparent !important; border: none !important; padding: 0 !important; margin: 0 !important;">{row["Name"]}</div>', # Basic styling with black text and no background/border/padding/margin
                    # Adjust icon_anchor to position the label relative to the location
                    # [0, 0] anchors the top-left of the div at the location
                    # [0, -10] would anchor above the location by 10 pixels
                    # Try adjusting anchor if needed for positioning relative to circle marker
                    icon_anchor=[0, 0] # Keep initial anchor, adjust if needed
                )
            )
            m.add_layer(label)

        else:
             # Optional: Handle cases where calculated_radius is negative (shouldn't happen with positive capacity but good practice)
             print(f"Skipping row {index} due to negative calculated radius: {calculated_radius}")
    else:
        # Optional: Handle rows where Capacity or Name is invalid
        print(f"Skipping row {index} due to invalid Capacity or Name: Capacity={row.get('Capacity')}, Name={row.get('Name')}")


display(m)

Map(center=[55.0, -3.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
from ipyleaflet import Map, CircleMarker, basemaps
import pandas as pd

# Calculate center using grouped data (this is no longer needed for centering on the UK, but keeping it for potential future use or if you still want the data's center as a reference)
# center = (df['Lat'].mean(), df['Long'].mean())

# Set the center and zoom for the UK
uk_center = (55.0, -3.0) # Approximate center of the UK
uk_zoom = 5            # Adjust zoom level as needed (e.g., 5 or 6)
height=7000

# Create the map with a different base layer
# Using CARTO's Positron map which is minimalist and grayscale/light
mgreen = Map(center=uk_center, zoom=uk_zoom, basemap=basemaps.NASAGIBS.BlueMarble, height=700)

# Ensure 'Capacity' is numeric after removing commas.
# The errors='coerce' in the previous pd.to_numeric call will result in NaN for invalid entries.
# We keep it as float to allow for NaN values.
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False).astype(float)

for index, row in df.iterrows():
    # Check if the 'Capacity' value is a valid number (not NaN)
    if pd.notna(row['Capacity']):
        # Calculate the radius, ensuring the result is a valid number before converting to int
        # Also ensure the result is non-negative, as radius must be positive
        # Adjusted divisor based on previous context
        calculated_radius = row['Capacity'] / 75 # Use 1000 as divisor again
        if calculated_radius >= 0:
            marker = CircleMarker(
                location=(row['Lat'], row['Long']),
                # Convert to integer only if the calculated radius is a valid non-negative number
                radius=int(calculated_radius),
                color=(row['Colour']),
                fill_color=(row['Colour']),
                fill_opacity=1
            )
            m.add_layer(marker)
        else:
             # Optional: Handle cases where calculated_radius is negative (shouldn't happen with positive capacity but good practice)
             print(f"Skipping row {index} due to negative calculated radius: {calculated_radius}")
    else:
        # Optional: Handle rows where Capacity is not a valid number (NaN)
        print(f"Skipping row {index} due to invalid Capacity: {row['Capacity']}")


display(mgreen)

Map(center=[55.0, -3.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
from ipyleaflet import Map, CircleMarker, basemaps
import pandas as pd

# Calculate center using grouped data (this is no longer needed for centering on the UK, but keeping it for potential future use or if you still want the data's center as a reference)
# center = (df['Lat'].mean(), df['Long'].mean())

# Set the center and zoom for the UK
uk_center = (55.0, -3.0) # Approximate center of the UK
uk_zoom = 5            # Adjust zoom level as needed (e.g., 5 or 6)
height=7000

# Create the map with a different base layer
# Using CARTO's Positron map which is minimalist and grayscale/light
mgrey = Map(center=uk_center, zoom=uk_zoom, basemap=basemaps.NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief, height=700)

# Ensure 'Capacity' is numeric after removing commas.
# The errors='coerce' in the previous pd.to_numeric call will result in NaN for invalid entries.
# We keep it as float to allow for NaN values.
df['Capacity'] = df['Capacity'].astype(str).str.replace(',', '', regex=False).astype(float)

for index, row in df.iterrows():
    # Check if the 'Capacity' value is a valid number (not NaN)
    if pd.notna(row['Capacity']):
        # Calculate the radius, ensuring the result is a valid number before converting to int
        # Also ensure the result is non-negative, as radius must be positive
        # Adjusted divisor based on previous context
        calculated_radius = row['Capacity'] / 75 # Use 1000 as divisor again
        if calculated_radius >= 0:
            marker = CircleMarker(
                location=(row['Lat'], row['Long']),
                # Convert to integer only if the calculated radius is a valid non-negative number
                radius=int(calculated_radius),
                color=(row['Colour']),
                fill_color=(row['Colour']),
                fill_opacity=1
            )
            m.add_layer(marker)
        else:
             # Optional: Handle cases where calculated_radius is negative (shouldn't happen with positive capacity but good practice)
             print(f"Skipping row {index} due to negative calculated radius: {calculated_radius}")
    else:
        # Optional: Handle rows where Capacity is not a valid number (NaN)
        print(f"Skipping row {index} due to invalid Capacity: {row['Capacity']}")


display(mgrey)

Map(center=[55.0, -3.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
# prompt: how to save a map to google drive

import os
from ipyleaflet import Map
import json

# Assuming 'm' is your ipyleaflet map object
# You cannot directly save the interactive ipyleaflet object as a static image or file easily.
# ipyleaflet maps are designed to be displayed within environments like Jupyter notebooks/Colab.
# To "save" it to Google Drive, you can consider:
# 1. Saving the data used to create the map (e.g., as a CSV, GeoJSON).
# 2. Taking a screenshot or using browser tools to capture the map display.
# 3. Saving the notebook itself which contains the map output (this notebook is already on Drive as it's a Colab notebook).

# If you want to save the data that generates the markers on the map, you can save the dataframe:
df.to_csv('/content/drive/MyDrive/map_data.csv', index=False)
print("Map data saved to Google Drive as map_data.csv")

# You could also potentially try saving the state of the ipyleaflet object,
# but loading and rendering it outside a notebook environment is not straightforward.
# Here's an example of trying to save the map object's state as a JSON,
# but this is for internal ipyleaflet representation, not a viewable file.
try:
    map_state = m.to_json()
    with open('/content/drive/MyDrive/map_state.json', 'w') as f:
        json.dump(map_state, f)
    print("Map state saved to Google Drive as map_state.json (for ipyleaflet use, not a viewable map file).")
except Exception as e:
    print(f"Could not save map state: {e}")
    print("Saving the interactive map directly as a simple file (like PNG or HTML) is not natively supported by ipyleaflet in this manner.")

